# Proyecto SQL

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

Te han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

# Objetivo del proyecto

* Analizar la base de datos para entender el comportamiento de los amantes de los libros y generar una propuesta para mejorar el servicio de lectura o crear un nuevo producto para satisfacer a la gente que le gusta leer en casa.

# Conexion a la base de datos

In [1]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Estudio de las tablas

In [2]:
books = "SELECT * FROM books;"
pd.io.sql.read_sql(books, con = engine).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
authors = "SELECT * FROM authors;"
pd.io.sql.read_sql(authors, con = engine).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
publishers = "SELECT * FROM publishers;"
pd.io.sql.read_sql(publishers, con = engine).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
ratings = "SELECT * FROM ratings;"
pd.io.sql.read_sql(ratings, con = engine).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
reviews = "SELECT * FROM reviews;"
pd.io.sql.read_sql(reviews, con = engine).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Analisis Exploratorio de Datos (EDA)

In [7]:
libros_publicados = """
SELECT COUNT(*) AS libros_publicados
FROM books
WHERE publication_date > '2000-01-01';
"""
pd.io.sql.read_sql(libros_publicados, con = engine)

,libros_publicados
0,819


**El numero de libros publicados despues del 1 de enero de 2000 es de 819 libros**

In [8]:
reseñas_y_calificaciones = """
SELECT
    b.book_id,
    b.title,
    COUNT(r.review_id) AS Reseñas,
    AVG(ra.rating) AS Promedio_calificacion
FROM
    books AS b
LEFT JOIN
    reviews AS r ON b.book_id = r.book_id
LEFT JOIN
    ratings AS ra ON b.book_id = ra.book_id
GROUP BY
    b.book_id, b.title
ORDER BY
    Reseñas DESC;
"""
pd.io.sql.read_sql(reseñas_y_calificaciones, con = engine).head()

,book_id,title,reseñas,promedio_calificacion
0,948,Twilight (Twilight #1),1120,3.662500
1,750,The Hobbit or There and Back Again,528,4.125000
2,673,The Catcher in the Rye,516,3.825581
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.414634
4,299,Harry Potter and the Chamber of Secrets (Harry...,480,4.287500


**En la anterior celda podemos ver el top 5 de libros con mas reseñas y con su respectiva calificacion**

In [9]:
editorial_mas_libros = """
SELECT 
  p.publisher, COUNT(b.book_id) AS total_libros
FROM 
  books AS b
JOIN 
  publishers AS p ON b.publisher_id = p.publisher_id
WHERE 
  num_pages > 50
GROUP BY 
  p.publisher
ORDER BY 
  total_libros DESC
"""
pd.io.sql.read_sql(editorial_mas_libros, con = engine).head()

,publisher,total_libros
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19


**En la celda anterior podemos ver la editorial que mas ha publicado libros con mas de 50 paginas, que es: Penguin Books**

In [10]:
autor_mas_calificado = """
SELECT 
  a.author, AVG(r.rating) AS promedio_calificacion
FROM 
  books AS b
JOIN 
  authors AS a ON b.author_id = a.author_id
JOIN 
  ratings AS r ON b.book_id = r.book_id
GROUP BY 
  a.author
HAVING 
  COUNT(r.rating_id) >= 50
ORDER BY 
  promedio_calificacion DESC
"""
pd.io.sql.read_sql(autor_mas_calificado, con = engine).head()

,author,promedio_calificacion
0,Diana Gabaldon,4.300000
1,J.K. Rowling/Mary GrandPré,4.288462
2,Agatha Christie,4.283019
3,Markus Zusak/Cao Xuân Việt Khương,4.264151
4,J.R.R. Tolkien,4.240964


**En la celda anterior podemos observar el autor con la mas alta calificacion promedio y que tiene mas de 50 calificaciones por usuario, que es: Diana Gabaldon**

In [11]:
promedio_reseñas = """
SELECT
  AVG(num_reviews) AS promedio_reseñas
  FROM (
    SELECT
      COUNT(r.text) AS num_reviews
    FROM
      reviews AS r
    WHERE
      r.username IN (
        SELECT
          username
        FROM
          ratings
        GROUP BY
          username
        HAVING
          COUNT(book_id) > 50
      )
    GROUP BY
      r.username
  )
  """
pd.io.sql.read_sql(promedio_reseñas, con = engine)


,promedio_reseñas
0,24.333333


**En la celda anterior podemos ver el promedio de reseñas de texto que hacen los usarios que calificaron mas de 50 libros**

<div class="alert alert-block alert-success">
<b>Paso 2</b> <a class="tocSkip"></a><br>
Has realizado un buen trabajo al formular una consulta SQL para contar con:
    
    - el número de libros publicados después de una fecha específica.
    - Calificación promedio
    - Editorial que ha publicado la mayor cantidad de libros
    - Autor con la calificación promedio más alta
    - Promedio de reseñas
    
Esto demuestra tu habilidad para manipular y consultar bases de datos de manera efectiva. Solamente para terminar de complementar te recomiendo incluir el número de libros evaluados por autor para proporcionar un contexto adicional sobre la consistencia de sus calificaciones.
</div>

# Conclusion

**Segun los resultados obtenidos podemos proponer:**
 
* **Un catálogo en constante crecimiento de libros electrónicos y audiolibros, con un enfoque especial en las novedades (como los 819 libros publicados después del 2000 identificados) y contenido de editoriales de mayor prestigio (como Penguin Books y Vintage). Esto asegura que los usuarios siempre tengan acceso a material fresco y de calidad.**

* **Un algoritmo de recomendación avanzado que sugiere libros basándose en los autores mejor calificados (ej. Diana Gabaldon, J.K. Rowling/Mary GrandPré) y el historial de lectura del usuario, garantizando que siempre encuentren su próxima lectura favorita.**

* **Un espacio dedicado para que los usuarios puedan escribir reseñas detalladas, calificar libros y discutir con otros lectores. Se potenciará la participación de usuarios altamente comprometidos (como los que promedian 24.33 reseñas), ofreciendo insignias o niveles para fomentar la interacción y la compartición de opiniones, capitalizando la importancia de las reseñas y calificaciones observadas.**